In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib_venn import venn2
from multiprocessing import Pool
import concurrent.futures
from concurrent.futures import ThreadPoolExecutor
import scipy.stats as ss
import re
import sys
sys.path.append("/home/ziryu_wang/.conda/envs/py36_cmap/lib/python3.6/site-packages")
#sys.path
from cmapPy.pandasGEXpress.parse import parse
import cmapPy.pandasGEXpress.subset_gctoo as sg

In [3]:
cp = parse("/home/ziryu_wang/database/cMAP/level5_beta_trt_cp_n720216x12328.gctx")
compoundinfo = pd.read_csv('/home/ziryu_wang/database/cMAP/compoundinfo_beta.txt',sep = '\t')
sigInfo = pd.read_csv('/home/ziryu_wang/database/cMAP/siginfo_beta.txt',sep='\t')
geneinfo = pd.read_csv('/home/ziryu_wang/database/cMAP/geneinfo_beta.txt', sep = '\t')
Celine = pd.read_csv('/home/ziryu_wang/database/cMAP/cellinfo_beta.txt',sep = '\t')
# Mapping TCGA acronyms to the dataset's primary disease entries
tcga_mapping = {
    "BLCA": "bladder cancer",
    "BRCA": "breast cancer",
    "COAD": "colon cancer",
    "HNSC": "head and neck cancer",
    "KIRC": "kidney cancer",
    "LIHC": "liver cancer",
    "LUAD": "lung cancer",
    "PRAD": "prostate cancer"
}

# Creating a DataFrame with this mapping
df_mapping = pd.DataFrame(list(tcga_mapping.items()), columns=['TCGA_Acronym', 'Primary_Disease'])

print(df_mapping)

/usr/local/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (0,3,4,28,29) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


  TCGA_Acronym       Primary_Disease
0         BLCA        bladder cancer
1         BRCA         breast cancer
2         COAD          colon cancer
3         HNSC  head and neck cancer
4         KIRC         kidney cancer
5         LIHC          liver cancer
6         LUAD           lung cancer
7         PRAD       prostate cancer


In [4]:
def process_data(cancertype):
    """
    Processes individual cancer type data by reading and transforming it.
    Returns transformed TCGA and immuAI dataframes for a given cancer type.
    """    
    
    tcga_path = f'/home/ziryu_wang/code_ava/Required_files/TCGA-{cancertype}_alldiff.csv'
    immuAI_path = f'/home/ziryu_wang/code_ava/Required_files/immuAI_{cancertype}.txt'

    # Read and process the TCGA data
    tcga = pd.read_csv(tcga_path, index_col=0)
    tcga = (tcga.iloc[:, 4:len(tcga.columns)-2]).T
    print(f'{cancertype} TCGA {tcga.shape[0]}')

    # Read the immuAI data
    immuAI = pd.read_csv(immuAI_path, sep='\t')

    return tcga, immuAI

def main():
    # Assume df_mapping is preloaded
    cancertypes = df_mapping['TCGA_Acronym']

    with Pool(processes=8) as pool:
        results = pool.map(process_data, cancertypes)

    # Unpack the results and concatenate all the dataframes
    all_merged_tcga, all_merged_immuAI = zip(*results)
    final_tcga = pd.concat(all_merged_tcga)
    final_immuAI = pd.concat(all_merged_immuAI)

    # Print the total number of samples processed
    print(f'All TCGA sample: {final_tcga.shape[0]}')
    print(f'All immuAI sample: {final_immuAI.shape[0]}')
    #final_tcga.to_csv('/home/ziryu_wang/code_ava/step3-cmapPy4_DurgIdent/output/final_tcga.csv')
    #final_immuAI.to_csv('/home/ziryu_wang/code_ava/step3-cmapPy4_DurgIdent/output/final_immuAI.csv')
    return final_tcga, final_immuAI


if __name__ == '__main__':
    final_tcga, final_immuAI = main()
    #final_tcga = pd.read_csv('/home/ziryu_wang/code_ava/step3-cmapPy4_DurgIdent/output/final_tcga.csv')
    #final_immuAI = pd.read_csv('/home/ziryu_wang/code_ava/step3-cmapPy4_DurgIdent/output/final_immuAI.csv')

    

LIHC TCGA 424
BLCA TCGA 433
COAD TCGA 521
LUAD TCGA 594
KIRC TCGA 611
PRAD TCGA 551
HNSC TCGA 546
BRCA TCGA 1222
All TCGA sample: 4902
All immuAI sample: 4696


In [6]:
## low dose
sel_compound = pd.read_csv(f'/home/ziryu_wang/code_ava/Required_files/drugCom54.csv',index_col=0)
sel_compound = sel_compound.T
sel_compound_filt = sel_compound.index
# Replacing non-zero values with column names
for column in sel_compound.columns:
    sel_compound[column] = sel_compound[column].apply(lambda x: column if x != 0 else 0)

# Initialize a list to store each DataFrame for genes
drug_stats = []

# Initialize a list to store each DataFrame
all_merged_dfs = []

# Iterate over each drug and print the corresponding cancer types
for drug in sel_compound_filt:
    cancertype = sel_compound.loc[drug, sel_compound.loc[drug] != 0].tolist()
    #celltype = df_mapping['Primary_Disease'].tolist()
    celltype = df_mapping[df_mapping['TCGA_Acronym'].isin(cancertype)]['Primary_Disease'].tolist()
    print(f"Drug: {drug}, celltype: {celltype}")
    coloncl = Celine[Celine['primary_disease'].isin(celltype)]
    my_col_metadata = parse("/home/ziryu_wang/database/cMAP/level5_beta_trt_cp_n720216x12328.gctx", col_meta_only=True)
    sigInfo1 = sigInfo[sigInfo['sig_id'].isin(my_col_metadata.index.values)]
    sigInfo1 = sigInfo1[(sigInfo1['pert_dose'] < sigInfo['pert_dose'].median()) & (sigInfo1['pert_time'] < 48.0)]#low dose
    colon_tab = sigInfo1[sigInfo1["cell_iname"].isin(coloncl["cell_iname"])]
    colon_ids = sigInfo1['sig_id'][sigInfo1['cmap_name'].isin([drug])]
    print(f'{len(colon_ids)} ids were found from {drug}.')

    mito_gctoo = parse("/home/ziryu_wang/database/cMAP/level5_beta_trt_cp_n720216x12328.gctx", cid=colon_ids)
    mito_gctoo.data_df.mask((mito_gctoo.data_df > -1.645) & (mito_gctoo.data_df < 1.645), np.nan, inplace=True)
    
    # Quantile calculation
    quantiles = mito_gctoo.data_df.quantile([0.33, 0.67], axis=1)
    max_quantile = quantiles.apply(lambda x: x[0.67] if abs(x[0.67]) > abs(x[0.33]) else x[0.33], axis=0)
    print(max_quantile)

    # Normalized using the greatest quantile
    normalized_df = (mito_gctoo.data_df - max_quantile.min()) / (max_quantile.max() - max_quantile.min())
    normalized_df = normalized_df.clip(lower=0, upper=1)  # Limit the value to between 0 and 1
    print(normalized_df)

    # Calculate the standardized mean score for each gene
    gene_normalized_scores = pd.DataFrame(normalized_df.mean(axis=1), columns=['normalized_score'])
    gene_normalized_scores['max_quantile']=max_quantile
    gene_normalized_scores['gene_id'] = gene_normalized_scores.index
    gene_normalized_scores['gene_id'] = gene_normalized_scores['gene_id'].astype(int)
    print(gene_normalized_scores)

    # Merge data
    gene_M = geneinfo[geneinfo['gene_id'].isin(gene_normalized_scores.index)]
    merged_df = pd.merge(gene_normalized_scores, gene_M, on='gene_id')
    merged_df['Drug'] = drug  # 添加药物列
    merged_df_fil = merged_df[(merged_df["max_quantile"] <= -3.891) | (merged_df["max_quantile"] >= 3.891)]#99.99% zscore
    all_merged_dfs.append(merged_df_fil)
    print(f'================================== {merged_df_fil.shape[0]} genes were found from {drug}.')
    
    # Append drug statistics
    drug_stats.append([drug, len(colon_ids), merged_df_fil.shape[0]])

# Concatenate all DataFrames into a single DataFrame
final_df = pd.concat(all_merged_dfs)
final_df.to_csv('/home/ziryu_wang/code_ava/step3-cmapPy4_DurgIdent/output/drug54ToGene.csv')

# Create and save statistics DataFrame
stats_df = pd.DataFrame(drug_stats, columns=['drug', 'number of pert_id', 'number of pert_genes'])
print(stats_df)
stats_df.to_csv('/home/ziryu_wang/code_ava/step3-cmapPy4_DurgIdent/output/drugStats-l.csv', index=False)


Drug: daunorubicin, celltype: ['bladder cancer', 'breast cancer', 'colon cancer', 'head and neck cancer', 'kidney cancer', 'liver cancer', 'lung cancer', 'prostate cancer']
393 ids were found from daunorubicin.
================================== 276 genes were found from daunorubicin.
Drug: alvocidib, celltype: ['bladder cancer', 'breast cancer', 'colon cancer', 'head and neck cancer', 'kidney cancer', 'liver cancer', 'lung cancer', 'prostate cancer']
490 ids were found from alvocidib.
================================== 2254 genes were found from alvocidib.
Drug: mitoxantrone, celltype: ['bladder cancer', 'breast cancer', 'colon cancer', 'head and neck cancer', 'kidney cancer', 'liver cancer', 'lung cancer', 'prostate cancer']
975 ids were found from mitoxantrone.
================================== 289 genes were found from mitoxantrone.
Drug: epirubicin, celltype: ['bladder cancer', 'breast cancer', 'colon cancer', 'head and neck cancer', 'kidney cancer', 'liver cancer', 'lung cancer'

45 ids were found from staurosporine.
================================== 1003 genes were found from staurosporine.
Drug: nepafenac, celltype: ['breast cancer', 'lung cancer', 'prostate cancer']
158 ids were found from nepafenac.
================================== 260 genes were found from nepafenac.
Drug: oxypurinol, celltype: ['breast cancer', 'colon cancer', 'lung cancer']
174 ids were found from oxypurinol.
================================== 206 genes were found from oxypurinol.
Drug: tizanidine, celltype: ['breast cancer', 'lung cancer', 'prostate cancer']
132 ids were found from tizanidine.
================================== 2657 genes were found from tizanidine.
Drug: wortmannin, celltype: ['breast cancer', 'colon cancer', 'prostate cancer']
691 ids were found from wortmannin.
================================== 32 genes were found from wortmannin.
Drug: AZD-5438, celltype: ['colon cancer', 'lung cancer', 'prostate cancer']
372 ids were found from AZD-5438.
=======================

In [7]:
##  high dose
sel_compound = pd.read_csv(f'/home/ziryu_wang/code_ava/Required_files/drugCom54.csv',index_col=0)
sel_compound = sel_compound.T
sel_compound_filt = sel_compound.index
# Replacing non-zero values with column names
for column in sel_compound.columns:
    sel_compound[column] = sel_compound[column].apply(lambda x: column if x != 0 else 0)

# Initialize a list to store each DataFrame for genes
drug_stats = []

# Initialize a list to store each DataFrame
all_merged_dfs = []

# Iterate over each drug and print the corresponding cancer types
for drug in sel_compound_filt:
    cancertype = sel_compound.loc[drug, sel_compound.loc[drug] != 0].tolist()
    #celltype = df_mapping['Primary_Disease'].tolist()
    celltype = df_mapping[df_mapping['TCGA_Acronym'].isin(cancertype)]['Primary_Disease'].tolist()
    print(f"Drug: {drug}, celltype: {celltype}")
    coloncl = Celine[Celine['primary_disease'].isin(celltype)]
    my_col_metadata = parse("/home/ziryu_wang/database/cMAP/level5_beta_trt_cp_n720216x12328.gctx", col_meta_only=True)
    sigInfo1 = sigInfo[sigInfo['sig_id'].isin(my_col_metadata.index.values)]
    sigInfo1 = sigInfo1[(sigInfo1['pert_dose'] > sigInfo['pert_dose'].median()) & (sigInfo1['pert_time'] >= 24)]#high dose
    colon_tab = sigInfo1[sigInfo1["cell_iname"].isin(coloncl["cell_iname"])]
    colon_ids = sigInfo1['sig_id'][sigInfo1['cmap_name'].isin([drug])]
    print(f'{len(colon_ids)} ids were found from {drug}.')

    mito_gctoo = parse("/home/ziryu_wang/database/cMAP/level5_beta_trt_cp_n720216x12328.gctx", cid=colon_ids)
    mito_gctoo.data_df.mask((mito_gctoo.data_df > -1.645) & (mito_gctoo.data_df < 1.645), np.nan, inplace=True)
    
    # Quantile calculation
    quantiles = mito_gctoo.data_df.quantile([0.33, 0.67], axis=1)
    max_quantile = quantiles.apply(lambda x: x[0.67] if abs(x[0.67]) > abs(x[0.33]) else x[0.33], axis=0)
    print(max_quantile)

    # Normalized using the greatest quantile
    normalized_df = (mito_gctoo.data_df - max_quantile.min()) / (max_quantile.max() - max_quantile.min())
    normalized_df = normalized_df.clip(lower=0, upper=1)  # Limit the value to between 0 and 1
    print(normalized_df)

    # Calculate the standardized mean score for each gene
    gene_normalized_scores = pd.DataFrame(normalized_df.mean(axis=1), columns=['normalized_score'])
    gene_normalized_scores['max_quantile']=max_quantile
    gene_normalized_scores['gene_id'] = gene_normalized_scores.index
    gene_normalized_scores['gene_id'] = gene_normalized_scores['gene_id'].astype(int)
    print(gene_normalized_scores)

    # Merge data
    gene_M = geneinfo[geneinfo['gene_id'].isin(gene_normalized_scores.index)]
    merged_df = pd.merge(gene_normalized_scores, gene_M, on='gene_id')
    merged_df['Drug'] = drug  # 添加药物列
    merged_df_fil = merged_df[(merged_df["max_quantile"] <= -3.891) | (merged_df["max_quantile"] >= 3.891)]#99.99% zscore
    all_merged_dfs.append(merged_df_fil)
    print(f'================================== {merged_df_fil.shape[0]} genes were found from {drug}.')
    
    # Append drug statistics
    drug_stats.append([drug, len(colon_ids), merged_df_fil.shape[0]])

# Concatenate all DataFrames into a single DataFrame
final_df = pd.concat(all_merged_dfs)
final_df.to_csv('/home/ziryu_wang/code_ava/step3-cmapPy4_DurgIdent/output/drug54ToGene-h.csv')

# Create and save statistics DataFrame
stats_df = pd.DataFrame(drug_stats, columns=['drug', 'number of pert_id', 'number of pert_genes'])
print(stats_df)
stats_df.to_csv('/home/ziryu_wang/code_ava/step3-cmapPy4_DurgIdent/output/drugStats-h.csv', index=False)


Drug: daunorubicin, celltype: ['bladder cancer', 'breast cancer', 'colon cancer', 'head and neck cancer', 'kidney cancer', 'liver cancer', 'lung cancer', 'prostate cancer']
159 ids were found from daunorubicin.
================================== 4641 genes were found from daunorubicin.
Drug: alvocidib, celltype: ['bladder cancer', 'breast cancer', 'colon cancer', 'head and neck cancer', 'kidney cancer', 'liver cancer', 'lung cancer', 'prostate cancer']
201 ids were found from alvocidib.
================================== 4404 genes were found from alvocidib.
Drug: mitoxantrone, celltype: ['bladder cancer', 'breast cancer', 'colon cancer', 'head and neck cancer', 'kidney cancer', 'liver cancer', 'lung cancer', 'prostate cancer']
628 ids were found from mitoxantrone.
================================== 1909 genes were found from mitoxantrone.
Drug: epirubicin, celltype: ['bladder cancer', 'breast cancer', 'colon cancer', 'head and neck cancer', 'kidney cancer', 'liver cancer', 'lung cance

28 ids were found from staurosporine.
================================== 2614 genes were found from staurosporine.
Drug: nepafenac, celltype: ['breast cancer', 'lung cancer', 'prostate cancer']
74 ids were found from nepafenac.
================================== 200 genes were found from nepafenac.
Drug: oxypurinol, celltype: ['breast cancer', 'colon cancer', 'lung cancer']
92 ids were found from oxypurinol.
================================== 3484 genes were found from oxypurinol.
Drug: tizanidine, celltype: ['breast cancer', 'lung cancer', 'prostate cancer']
34 ids were found from tizanidine.
================================== 129 genes were found from tizanidine.
Drug: wortmannin, celltype: ['breast cancer', 'colon cancer', 'prostate cancer']
600 ids were found from wortmannin.
================================== 48 genes were found from wortmannin.
Drug: AZD-5438, celltype: ['colon cancer', 'lung cancer', 'prostate cancer']
77 ids were found from AZD-5438.
===========================

In [8]:
## final
import concurrent.futures
from concurrent.futures import ThreadPoolExecutor

    
def process_drug(drug, final_tcga, final_df, final_immuAI):
    # This function integrates the previously separated steps into one function per drug
    mixdata, df_sel = filter_and_merge_data(drug, final_tcga, final_df, final_immuAI)
    save_rho_pval = calculate_correlations(mixdata)
    cor = analyze_correlations(df_sel, save_rho_pval)
    drugimm1 = select_significant_correlations(cor)
    return drug, drugimm1

def filter_and_merge_data(drug, final_tcga, final_df, final_immuAI):
    print(drug)
    #print(f'Gene all: {final_tcga.shape[1]}')
    
    # Select genes based on the drug
    df_sel = final_df.loc[final_df['Drug'].isin([drug]),:]
    #print(f'==========df_sel {df_sel}')
    final_tcga_fil = final_tcga.loc[:, final_tcga.columns.isin(df_sel['ensembl_id'])]
    print(f'Gene selected by {drug}: {final_tcga_fil.shape[1]}')
    #print(f'==========final_tcga_fil {final_tcga_fil}')
    
    # Preparing data for merging
    final_tcga_fil['sample'] = final_tcga_fil.index
    final_immuAI.index = final_immuAI.index.str.replace('.','-')
    final_immuAI['sample'] = final_immuAI.index
    mixdata = pd.merge(final_immuAI, final_tcga_fil, on='sample')
    mixdata = mixdata[mixdata['sample'].str.contains('.*-0[1-9]A-')]
    mixdata = mixdata.drop(columns=['sample'])
    mixdata = mixdata.dropna(axis=1, how='any')
    
    return mixdata, df_sel

def calculate_correlations(mixdata):
    rho, pval = ss.spearmanr(mixdata)
    save_rho = pd.DataFrame(rho, columns=mixdata.columns, index=mixdata.columns)
    save_pval = pd.DataFrame(pval, columns=mixdata.columns, index=mixdata.columns)   
    #print(f'=========save_pval {drug} {save_pval}')   
    save_pval[save_pval >= 0.05] = np.nan
    save_pval[save_pval < 0.05] = 0
          
    save_rho_pval = save_rho.iloc[25:len(mixdata.columns), 0:25] + save_pval.iloc[25:len(mixdata.columns), 0:25]
    #print(f'==========len(save_rho_pval):{drug} {len(save_rho_pval)}')
    #print(f'{drug} {save_rho_pval}')
    return save_rho_pval

def analyze_correlations(df_sel, save_rho_pval):
    upG = df_sel.loc[df_sel['max_quantile'] >= 1.96, 'ensembl_id']
    downG = df_sel.loc[df_sel['max_quantile'] <= -1.96, 'ensembl_id']
    
    downGcor = -save_rho_pval[save_rho_pval.index.isin(downG)]
    upGcor = save_rho_pval[save_rho_pval.index.isin(upG)]
    cor = pd.concat([downGcor, upGcor])
    
    cor.loc["col_med"] = cor.apply(lambda x: x.median(), axis=0)
    cor.loc['gene_count'] = len(save_rho_pval) - cor.isna().sum()
    #print(f'==========len(cor):{drug} {len(cor)}')
    return cor

def select_significant_correlations(cor):
    drugimm = cor.loc[['col_med', 'gene_count']].T
    thre = 0.80*(len(cor)-2)
    #print(f'==========80% of selected genes contribute to the immune cell infiltrate (thre): {drug}, {thre}')
    drugimm['significant'] = drugimm.apply(lambda row: row['col_med'] if row['gene_count'] > thre else 0, axis=1)    
    print(drugimm)
    return drugimm['significant']

def main(sel_compound_filt, final_tcga, final_df, final_immuAI):
    results = {}
    with ThreadPoolExecutor(max_workers=16) as executor:
        future_to_drug = {executor.submit(process_drug, drug, final_tcga, final_df, final_immuAI): drug for drug in sel_compound_filt}
        for future in concurrent.futures.as_completed(future_to_drug):
            drug = future_to_drug[future]
            try:
                drug_name, drugimm1 = future.result()
                results[drug_name] = drugimm1
            except Exception as exc:
                print(f'{drug} generated an exception: {exc}')

    # Combine all results into a single DataFrame
    combined_results = pd.DataFrame(results)
    print(combined_results) 
    combined_results.to_csv('/home/ziryu_wang/code_ava/step3-cmapPy4_DurgIdent/output/DTC_combined_drugs-h.csv')

if __name__ == '__main__':
    main(sel_compound_filt, final_tcga, final_df, final_immuAI)


daunorubicin
alvocidib
mitoxantrone
epirubicin
idarubicin
JNJ-7706621
bortezomib
panobinostat
MG-132
YM-155
doxorubicin
TP-0903
PF-562271
PHA-793887
BRD-A19037878
ponatinib
Gene selected by YM-155: 3953


<ipython-input-8-99815b7c30fb>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_tcga_fil['sample'] = final_tcga_fil.index


Gene selected by PF-562271: 1865
Gene selected by TP-0903: 6541
Gene selected by alvocidib: 4373
Gene selected by idarubicin: 5627Gene selected by JNJ-7706621: 3353

Gene selected by ponatinib: 4127
Gene selected by daunorubicin: 4604
Gene selected by bortezomib: 755
Gene selected by BRD-A19037878: 674
Gene selected by doxorubicin: 2209
Gene selected by MG-132: 729
Gene selected by epirubicin: 4893
Gene selected by panobinostat: 905
Gene selected by PHA-793887: 4067
Gene selected by mitoxantrone: 1889
                    col_med  gene_count  significant
CD4_naive          0.054868       584.0     0.054868
CD8_naive         -0.049564       527.0     0.000000
Tc                -0.076477       603.0    -0.076477
Tex               -0.079292       585.0    -0.079292
Tr1                0.052383       624.0     0.052383
nTreg             -0.041931       530.0     0.000000
iTreg             -0.062378       573.0    -0.062378
Th1               -0.073703       563.0    -0.073703
Th2             

<ipython-input-8-99815b7c30fb>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_tcga_fil['sample'] = final_tcga_fil.index


                    col_med  gene_count  significant
CD4_naive         -0.045892       634.0    -0.045892
CD8_naive         -0.049597       601.0    -0.049597
Tc                 0.049171       642.0     0.049171
Tex               -0.057535       632.0    -0.057535
Tr1               -0.040217       679.0    -0.040217
nTreg              0.048624       567.0     0.000000
iTreg              0.040743       617.0     0.040743
Th1               -0.054468       606.0    -0.054468
Th2                0.052244       634.0     0.052244
Th17              -0.031692       556.0     0.000000
Tfh                0.070066       589.0     0.070066
Tcm                0.095104       674.0     0.095104
Tem               -0.049925       615.0    -0.049925
NKT                0.056606       599.0     0.056606
MAIT              -0.059348       657.0    -0.059348
DC                -0.062972       635.0    -0.062972
B_cell            -0.051244       596.0    -0.051244
Monocyte          -0.039845       640.0    -0.

...

                    col_med  gene_count  significant
CD4_naive         -0.079410      1079.0    -0.079410
CD8_naive         -0.063877      1056.0    -0.063877
Tc                 0.069546      1106.0     0.069546
Tex               -0.064845      1070.0    -0.064845
Tr1               -0.092158      1146.0    -0.092158
nTreg              0.062487      1052.0     0.062487
iTreg              0.057489      1039.0     0.000000
Th1               -0.068264      1032.0     0.000000
Th2                0.128330      1149.0     0.128330
Th17              -0.061601       969.0     0.000000
Tfh                0.103630      1093.0     0.103630
Tcm                0.158838      1171.0     0.158838
Tem               -0.070939      1063.0    -0.070939
NKT                0.087086      1045.0     0.087086
MAIT              -0.112724      1153.0    -0.112724
DC                -0.089922      1141.0    -0.089922
B_cell            -0.046836       977.0     0.000000
Monocyte          -0.056550      1100.0    -0.

<ipython-input-8-99815b7c30fb>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_tcga_fil['sample'] = final_tcga_fil.index


                    col_med  gene_count  significant
CD4_naive         -0.049333       584.0    -0.049333
CD8_naive         -0.058833       518.0     0.000000
Tc                 0.040359       570.0     0.040359
Tex               -0.038089       549.0    -0.038089
Tr1               -0.046307       589.0    -0.046307
nTreg              0.036485       513.0     0.000000
iTreg              0.034173       531.0     0.034173
Th1               -0.064271       530.0    -0.064271
Th2                0.057673       571.0     0.057673
Th17              -0.049174       507.0     0.000000
Tfh                0.077952       547.0     0.077952
Tcm                0.087670       584.0     0.087670
Tem               -0.048340       540.0    -0.048340
NKT                0.085113       526.0     0.085113
MAIT              -0.076685       581.0    -0.076685
DC                -0.065539       565.0    -0.065539
B_cell            -0.047367       502.0     0.000000
Monocyte          -0.076551       563.0    -0.

                    col_med  gene_count  significant
CD4_naive         -0.082891      2176.0    -0.082891
CD8_naive         -0.058947      2063.0     0.000000
Tc                 0.071468      2237.0     0.071468
Tex               -0.043525      2138.0    -0.043525
Tr1               -0.063923      2304.0    -0.063923
nTreg              0.059768      2102.0     0.059768
iTreg              0.076793      2104.0     0.076793
Th1               -0.053233      2086.0    -0.053233
Th2                0.084978      2249.0     0.084978
Th17              -0.056809      1914.0     0.000000
Tfh                0.103851      2165.0     0.103851
Tcm                0.158649      2322.0     0.158649
Tem               -0.076505      2151.0    -0.076505
NKT                0.071958      2035.0     0.000000
MAIT              -0.112868      2248.0    -0.112868
DC                -0.050234      2243.0    -0.050234
B_cell            -0.035810      1989.0     0.000000
Monocyte          -0.055622      2187.0    -0.

                    col_med  gene_count  significant
CD4_naive         -0.080363      3271.0    -0.080363
CD8_naive         -0.049775      3021.0     0.000000
Tc                 0.078455      3398.0     0.078455
Tex               -0.072647      3303.0    -0.072647
Tr1               -0.107421      3524.0    -0.107421
nTreg              0.057504      3154.0     0.057504
iTreg              0.056221      3132.0     0.056221
Th1               -0.060776      3166.0    -0.060776
Th2                0.114113      3460.0     0.114113
Th17              -0.050358      2790.0     0.000000
Tfh                0.091556      3208.0     0.091556
Tcm                0.154136      3518.0     0.154136
Tem               -0.053711      3184.0    -0.053711
NKT                0.061385      3034.0     0.000000
MAIT              -0.117142      3463.0    -0.117142
DC                -0.096119      3378.0    -0.096119
B_cell            -0.046940      2987.0     0.000000
Monocyte          -0.043258      3308.0    -0.